<a href="https://colab.research.google.com/github/rifathaqueamit/mythesistest/blob/develop/Action_recognition_in_video_(Samsung).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the Google Colab for demonstrating the action recognition inside video.

Install necessary dependencies

In [ ]:
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.0 MB/s eta 0:00:00


Imports

In [ ]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import mediapy as media
import numpy as np
import PIL
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tqdm
import absl.logging
import tempfile
import ssl
import re
from urllib import request  # requires python3
import cv2

tf.get_logger().setLevel('ERROR')
absl.logging.set_verbosity(absl.logging.ERROR)
mpl.rcParams.update({
    'font.size': 10,
})

In [ ]:
!mkdir output_videos

mkdir: cannot create directory ‘output_videos’: File exists


Load pre-trained model

In [ ]:
tflite_filename = 'model.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_filename)
runner = interpreter.get_signature_runner()

Utility

In [ ]:
# Download Kinetics 600 label map
!wget https://raw.githubusercontent.com/tensorflow/models/f8af2291cced43fc9f1d9b41ddbf772ae7b0d7d2/official/projects/movinet/files/kinetics_600_labels.txt -O labels.txt -q

with tf.io.gfile.GFile('labels.txt') as f:
  lines = f.readlines()
  KINETICS_600_LABELS_LIST = [line.strip() for line in lines]
  KINETICS_600_LABELS = tf.constant(KINETICS_600_LABELS_LIST)

def get_top_k(probs, k=5, label_map=KINETICS_600_LABELS):
  top_predictions = tf.argsort(probs, axis=-1, direction='DESCENDING')[:k]
  top_labels = tf.gather(label_map, top_predictions, axis=-1)
  top_labels = [label.decode('utf8') for label in top_labels.numpy()]
  top_probs = tf.gather(probs, top_predictions, axis=-1).numpy()
  return tuple(zip(top_labels, top_probs))

Load input video and convert it to suitable format

In [ ]:
!ffmpeg -i 20230714_134853.mp4 -vf "scale=172:172:force_original_aspect_ratio=decrease,fps=5,pad=172:172:(ow-iw)/2:(oh-ih)/2,setsar=1" output_video.avi

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
!ffmpeg -i 20230714_134853.mp4 -vf "scale=172:172,fps=5,pad=172:172:(ow-iw)/2:(oh-ih)/2,setsar=1" output_video.avi

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
def crop_center_square(frame):
  y, x = frame.shape[0:2]
  min_dim = min(y, x)
  start_x = (x // 2) - (min_dim // 2)
  start_y = (y // 2) - (min_dim // 2)
  return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=0, resize=(172, 172)):
  cap = cv2.VideoCapture(path)
  frames = []
  try:
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      # frame = crop_center_square(frame)
      #  frame = cv2.resize(frame, resize)
      frame = frame[:, :, [2, 1, 0]]
      frames.append(frame)

      if len(frames) == max_frames:
        break
  finally:
    cap.release()
  return np.array(frames) / 255.0

Play the video that is converted

In [ ]:
video_file = 'output_video.avi'
video_to_show = media.read_video(video_file)
print(f'Shape is (num_images, height, width, num_channels) = {video_to_show.shape}.')
print(f'Framerate is {video_to_show.metadata.fps} frames/s.')
media.show_video(video_to_show)

video = load_video(video_file)
video = tf.cast(video, tf.float32)

Shape is (num_images, height, width, num_channels) = (157, 172, 172, 3).
Framerate is 5.0 frames/s.


This browser does not support the video tag.


Run the inference on every 30 seconds of clip

In [ ]:
init_states = {
    name: tf.zeros(x['shape'], dtype=x['dtype'])
    for name, x in runner.get_input_details().items()
}
del init_states['image']

In [ ]:
# make video chunks
chunks = []
single_chunk = []
for frame in video:
  if len(single_chunk) < 30:
    single_chunk.append(frame)
  else:
    chunks.append(single_chunk)
    single_chunk.clear()

for chunk in chunks:
  init_states = {
    name: tf.zeros(x['shape'], dtype=x['dtype'])
    for name, x in runner.get_input_details().items()
  }
  del init_states['image']

  states = init_states
  for clip in chunk:
    outputs = runner(**states, image=clip)
    logits = outputs.pop('logits')[0]
    states = outputs

  probs = tf.nn.softmax(logits)
  top_k = get_top_k(probs)
  print()
  for label, prob in top_k:
    print(label, prob)


kicking soccer ball 0.2552907
luge 0.1513462
throwing ball (not baseball or American football) 0.11482046
using a sledge hammer 0.06975892
passing american football (not in game) 0.061121587

kicking soccer ball 0.2552907
luge 0.1513462
throwing ball (not baseball or American football) 0.11482046
using a sledge hammer 0.06975892
passing american football (not in game) 0.061121587

kicking soccer ball 0.2552907
luge 0.1513462
throwing ball (not baseball or American football) 0.11482046
using a sledge hammer 0.06975892
passing american football (not in game) 0.061121587

kicking soccer ball 0.2552907
luge 0.1513462
throwing ball (not baseball or American football) 0.11482046
using a sledge hammer 0.06975892
passing american football (not in game) 0.061121587

kicking soccer ball 0.2552907
luge 0.1513462
throwing ball (not baseball or American football) 0.11482046
using a sledge hammer 0.06975892
passing american football (not in game) 0.061121587
